In [59]:
# Import all the needed libraries
import numpy as np
import matplotlib as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [60]:
# Let's get a grasp on the training data
train_data = pd.read_csv('train.csv')
print(train_data.head())
print(train_data.shape)

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [61]:
# Let's get a grasp on the testing data
test_data = pd.read_csv('test.csv')
print(test_data.head())
print(test_data.shape)

   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel9    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0   

In [62]:
# Compare test and training dataset
print(test_data.shape[0] / (train_data.shape[0] + test_data.shape[0]))

0.4


In [63]:
# Seperate pixels from label
X_train = train_data.drop(['label'], axis=1)
y_train = train_data[['label']]
X_test = test_data

In [64]:
def standardize(data):
    data / 255.0

In [65]:
X_train, X_test = X_train / 255.0, X_test / 255.0

In [66]:
np.array(X_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [81]:
from keras.models import Sequential
from keras.layers.core import Lambda, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D
def get_cnn_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28, 28, 1)),
        Convolution2D(32, (3, 3), activation='relu'),
        Convolution2D(32, (3, 3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64, (3, 3), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='relu')
    ])
    model.compile(Adam(), loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

model_sequential = get_cnn_model()
model.optimizer.lr=0.1

history = model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1,
                             validation_data=val_batches, validation_steps=val_batches.n)

AttributeError: 'NoneType' object has no attribute 'get_shape'

In [ ]:
results = model_sequential.predict_classes(X_test, verbose=1)
results_output = pd.DataFrame()
results_output['Label'] = results
results_output.index.name = 'ImageId'
results_output.index += 1
results_output.to_csv('sequentialModel.csv')

In [ ]:
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
def cnn_tf(X_train, y_train):
   

    cnn1 = tf.layers.conv2d(
            inputs = X_train,
            filters = 32,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu
        )

    pool1 = tf.layers.max_pooling2d(
            inputs = cnn1,
            pool_size=[2, 2],
            strides=2
        )

    cnn2 = tf.layers.conv2d(
            inputs = pool1,
            filters = 64,
            kernel_size = [5, 5],
            padding = "same",
            activation = tf.nn.relu
        )

    pool2 = tf.layers.max_pooling2d(
            inputs = cnn2,
            pool_size = [2, 2],
            strides=2
        )

    pool2_flatten = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flatten, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.4,
        training = 1
        )

    # Predictions
    logits = tf.layers.dense(inputs=dropout, units=1)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

#     print(y_train)
    print(logits)
    
    loss = tf.reduce_mean(tf.losses.log_loss(y_train, logits))
    train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    return train

In [ ]:
cnn_tf(X_train, y_train)

In [ ]:
with tf.Session():
    tensor = cnn_tf(X_train, y_train)
    tf.Print(tensor, [tensor])